### neural network implemented with word2vec only because tf-idf is a sparse matrix and we can't use it to train the nn

In [1]:
import pandas as pd
import numpy as np

import string
from re import sub

import matplotlib.pyplot as plt
import seaborn as sns

# from sklearn.preprocessing import LabelEncoder
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


from keras.layers import Input

import pickle
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

from sklearn.metrics import confusion_matrix


from sklearn.model_selection import cross_val_score
import pandas
import numpy

from sklearn.model_selection import cross_validate

from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

from sklearn.preprocessing import StandardScaler, MinMaxScaler


import tensorflow as tf



In [2]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1050 with Max-Q Design, pci bus id: 0000:02:00.0, compute capability: 6.1



### one fourth of the dataset

In [3]:
callback = EarlyStopping(monitor='val_loss',patience=2, min_delta=0.0001)

# checkpoint = ModelCheckpoint("best_model_conv", monitor='loss', verbose=1, save_best_only=True, mode='min')

reduce = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.5, min_lr=0.00001)

In [7]:
y_train, _, x_train = pickle.load(open("dbpedia_csv/" + "/train_vec_preprocessed_vectorized.pickle", "rb"))
#_, _, x_valid = pickle.load(open(paths.data + "/validation_preprocessed.pickle", "rb"))
y_test, _, x_test = pickle.load(open("dbpedia_csv/" + "/test_vec_preprocessed_vectorized.pickle", "rb"))


y_train = y_train[slice(0, len(y_train), 4)]
x_train = x_train[slice(0, len(x_train), 4)]


x_train_temp = pd.DataFrame(x_train).to_numpy()
y_train_temp = []


x_valid = []
y_valid = []

x_train = []
y_train_np = []


for i in range(len(y_train)):
    if i % 10 == 0:
        y_valid.append(y_train[i])
        x_valid.append(x_train_temp[i])
    else:
        y_train_np.append(y_train[i])
        x_train.append(x_train_temp[i])
        
y_train = y_train_np
x_train = np.array(x_train)
x_valid = np.array(x_valid)
    


# x_test_1 = x_test[slice(0, len(x_test), 2)]
# x_test_2 = x_test[slice(1, len(x_test), 2)]
# y_test_1 = y_test[slice(0, len(y_test), 2)]
# y_test_2 = y_test[slice(1, len(y_test), 2)]


# y_train_valid = y_train[slice(0, len(y_train), 10)]
# x_train_valid = x_train[slice(0, len(x_train), 10)]





# # list1 = [ elem for i,elem in enumerate(list1) if elem % 2 != 0]
# x_train = [e for i, e in enumerate(x_train) if i % 10 != 0]
# y_train = [e for i, e in enumerate(y_train) if i % 10 != 0]

# x_train_temp = [][]
# y_train_temp = []

# for i in range(1, 10):
#     x_train_temp += x_train[slice(i, len(x_train), 10)]
#     y_train_temp.append(y_train[i])
        

# print(len(x_train))


y_train_valid_np = []

for y in y_valid:
    l = [0 for x in range(14)]
    l[y-1] = 1
    y_train_valid_np.append(l)
    
y_valid = np.asarray(y_train_valid_np)
print(y_valid.shape)



y_test_np = []

for y in y_test:
    l = [0 for x in range(14)]
    l[y-1] = 1
    y_test_np.append(l)
    
y_test = np.asarray(y_test_np)
print(y_test.shape)


y_train_np = []

for y in y_train:
    l = [0 for x in range(14)]
    l[y-1] = 1
    y_train_np.append(l)
    
y_train = np.asarray(y_train_np)




PredictorScaler=MinMaxScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(x_train)


x_train = PredictorScalerFit.transform(x_train)
x_train_valid = PredictorScalerFit.transform(x_valid)
x_test = PredictorScalerFit.transform(x_test)


del y_train_np
del y_test_np
del y_train_valid_np
del x_train_temp
del PredictorScalerFit
del PredictorScaler
import sys
import gc
gc.collect()

## TRAIN AND FIT CLASSIFIER

t = time()

model_l1 = Sequential()
# model_l1.add(Embedding(56000, 1000))
model_l1.add(Input(shape=(1000,), name='input'))
# model_l1.add(Conv1D(128, 5, activation='relu'))
# model_l1.add(GlobalMaxPooling1D()) # added
model_l1.add(Dense(128, activation='relu'))
model_l1.add(Dropout(0.1))
model_l1.add(Dense(32, activation='relu'))
model_l1.add(Dense(14, activation='softmax')) 
model_l1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_l1.summary()

history_l1 = model_l1.fit(x_train, y_train, epochs=10, verbose=1, shuffle=True, use_multiprocessing=True, 
                   validation_data=(x_valid, y_valid), batch_size=32
                   ,callbacks=[reduce,callback])




training_time = time() - t
print("train time: %0.3fs" % training_time)

gc.collect()

t = time()
y_pred = model_l1.predict(x_test)

test_time = time() - t
print("test time:  %0.3fs" % test_time)

print(y_pred)

y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(y_test, axis=1)
# cm = confusion_matrix(y_test, y_pred)
# print(cm)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred, target_names=[str(x) for x in range(14)]))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')


(14000, 14)
(70000, 14)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               128128    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 32)                4128      
                                                                 
 dense_5 (Dense)             (None, 14)                462       
                                                                 
Total params: 132,718
Trainable params: 132,718
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
3938/3938 [==============================] - 82s 19ms/step - loss: 0.7082 - accuracy: 0.7700 - val_loss: 0.6665 - val_accuracy: 0.9184 - lr: 0.00

### half of the dataset

In [9]:
y_train, _, x_train = pickle.load(open("dbpedia_csv/" + "/train_vec_preprocessed_vectorized.pickle", "rb"))
#_, _, x_valid = pickle.load(open(paths.data + "/validation_preprocessed.pickle", "rb"))
y_test, _, x_test = pickle.load(open("dbpedia_csv/" + "/test_vec_preprocessed_vectorized.pickle", "rb"))


y_train = y_train[slice(0, len(y_train), 2)]
x_train = x_train[slice(0, len(x_train), 2)]


x_train_temp = pd.DataFrame(x_train).to_numpy()
y_train_temp = []


x_valid = []
y_valid = []

x_train = []
y_train_np = []


for i in range(len(y_train)):
    if i % 10 == 0:
        y_valid.append(y_train[i])
        x_valid.append(x_train_temp[i])
    else:
        y_train_np.append(y_train[i])
        x_train.append(x_train_temp[i])
        
y_train = y_train_np
x_train = np.array(x_train)
x_valid = np.array(x_valid)
    


y_test_np = []

for y in y_test:
    l = [0 for x in range(14)]
    l[y-1] = 1
    y_test_np.append(l)
    
y_test = np.asarray(y_test_np)
print(y_test.shape)

y_train_np = []

for y in y_train:
    l = [0 for x in range(14)]
    l[y-1] = 1
    y_train_np.append(l)
    
y_train = np.asarray(y_train_np)
print(y_train.shape)



y_train_valid_np = []

for y in y_valid:
    l = [0 for x in range(14)]
    l[y-1] = 1
    y_train_valid_np.append(l)
    
y_valid = np.asarray(y_train_valid_np)
print(y_valid.shape)




PredictorScaler=MinMaxScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(x_train)


x_train = PredictorScalerFit.transform(x_train)
x_test = PredictorScalerFit.transform(x_test)
x_valid = PredictorScalerFit.transform(x_valid)


del y_train_np
del y_test_np
del y_train_valid_np
del x_train_temp
del PredictorScalerFit
del PredictorScaler
import sys
import gc
gc.collect()

## TRAIN AND FIT CLASSIFIER

t = time()

model_l1 = Sequential()
# model_l1.add(Embedding(56000, 1000))
model_l1.add(Input(shape=(1000,), name='input'))
# model_l1.add(Conv1D(128, 5, activation='relu'))
# model_l1.add(GlobalMaxPooling1D()) # added
model_l1.add(Dense(128, activation='relu'))
model_l1.add(Dropout(0.1))
model_l1.add(Dense(32, activation='relu'))
model_l1.add(Dense(14, activation='softmax')) 
model_l1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_l1.summary()

history_l1 = model_l1.fit(x_train, y_train, epochs=10, verbose=1, use_multiprocessing=True,
                   validation_data=(x_valid, y_valid),
                    batch_size=32
                   ,callbacks=[reduce,callback])




training_time = time() - t
print("train time: %0.3fs" % training_time)

gc.collect()

t = time()
y_pred = model_l1.predict(x_test)



test_time = time() - t
print("test time:  %0.3fs" % test_time)

y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(y_test, axis=1)
# cm = confusion_matrix(y_test, y_pred)
# print(cm)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred, target_names=[str(x) for x in range(14)]))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')


(70000, 14)
(252000, 14)
(28000, 14)
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 128)               128128    
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 32)                4128      
                                                                 
 dense_11 (Dense)            (None, 14)                462       
                                                                 
Total params: 132,718
Trainable params: 132,718
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
7875/7875 [==============================] - 130s 16ms/step - loss: 0.6784 - accuracy: 0.7777 - val_loss: 0.2716 - val_accuracy: 0.9

### full dataset

In [4]:
y_train, _, x_train = pickle.load(open("dbpedia_csv/" + "/train_vec_preprocessed_vectorized.pickle", "rb"))
#_, _, x_valid = pickle.load(open(paths.data + "/validation_preprocessed.pickle", "rb"))
y_test, _, x_test = pickle.load(open("dbpedia_csv/" + "/test_vec_preprocessed_vectorized.pickle", "rb"))





x_train_temp = pd.DataFrame(x_train).to_numpy()
y_train_temp = []


x_valid = []
y_valid = []

x_train = []
y_train_np = []


for i in range(len(y_train)):
    if i % 10 == 0:
        y_valid.append(y_train[i])
        x_valid.append(x_train_temp[i])
    else:
        y_train_np.append(y_train[i])
        x_train.append(x_train_temp[i])
        
y_train = y_train_np
x_train = np.array(x_train)
x_valid = np.array(x_valid)
    




y_test_np = []

for y in y_test:
    l = [0 for x in range(14)]
    l[y-1] = 1
    y_test_np.append(l)
    
y_test = np.asarray(y_test_np)
print(y_test.shape)

y_train_np = []

for y in y_train:
    l = [0 for x in range(14)]
    l[y-1] = 1
    y_train_np.append(l)
    
y_train = np.asarray(y_train_np)
print(y_train.shape)


y_train_valid_np = []

for y in y_valid:
    l = [0 for x in range(14)]
    l[y-1] = 1
    y_train_valid_np.append(l)
    
y_valid = np.asarray(y_train_valid_np)
print(y_valid.shape)





PredictorScaler=MinMaxScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(x_train)


x_train = PredictorScalerFit.transform(x_train)
x_test = PredictorScalerFit.transform(x_test)
x_valid = PredictorScalerFit.transform(x_valid)


del y_train_np
del y_test_np
del y_train_valid_np
del x_train_temp
del PredictorScalerFit
del PredictorScaler
import sys
import gc
gc.collect()

## TRAIN AND FIT CLASSIFIER

t = time()

model_l1 = Sequential()
# model_l1.add(Embedding(56000, 1000))
model_l1.add(Input(shape=(1000,), name='input'))
# model_l1.add(Conv1D(128, 5, activation='relu'))
# model_l1.add(GlobalMaxPooling1D()) # added
model_l1.add(Dense(128, activation='relu'))
model_l1.add(Dropout(0.1))
model_l1.add(Dense(32, activation='relu'))
model_l1.add(Dense(14, activation='softmax')) 
model_l1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_l1.summary()

history_l1 = model_l1.fit(x_train, y_train, epochs=10, verbose=1, use_multiprocessing=True,
                   validation_data=(x_valid, y_valid),
                    batch_size=32
                   ,callbacks=[reduce,callback])



training_time = time() - t
print("train time: %0.3fs" % training_time)

gc.collect()

t = time()
y_pred = model_l1.predict(x_test)

test_time = time() - t
print("test time:  %0.3fs" % test_time)

y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(y_test, axis=1)
# cm = confusion_matrix(y_test, y_pred)
# print(cm)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred, target_names=[str(x) for x in range(14)]))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')


(70000, 14)
(504000, 14)
(56000, 14)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               128128    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 32)                4128      
                                                                 
 dense_2 (Dense)             (None, 14)                462       
                                                                 
Total params: 132,718
Trainable params: 132,718
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
15750/15750 [==============================] - 255s 15ms/step - loss: 0.4784 - accuracy: 0.8444 - val_loss: 0.2052 - val_accuracy: 0.9